In [6]:
OPENAI_API_KEY = ""
MODEL_NAME="gpt-3.5-turbo"


In [7]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 言語モデルの準備
chat_model = ChatOpenAI(
        openai_api_key=OPENAI_API_KEY,
        model=MODEL_NAME
)

In [23]:
class Chat:
    def __init__(self):
        self.store = {}
        self.runnable_with_history = None
        
    # システムプロンプトを定義する関数
    def create_system_prompt(self):
        return "あなたは親切で知識豊富なアシスタントです。ユーザーの質問に的確かつ簡潔に答えてください。"
    
    # プロンプトテンプレートを定義する関数
    def create_prompt_template(self):
        return ChatPromptTemplate.from_messages(
            [
                ("system", self.create_system_prompt()),  # システムプロンプトを追加
                MessagesPlaceholder(variable_name="history"),  # 会話履歴の追加
                ("human", "{input}"),
            ]
        )
        
    # セッションIDごとの会話履歴の取得
    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        return self.store[session_id]

    # RunnableWithMessageHistoryの準備
    def gen_runnable(self, session_id):
        prompt_template = self.create_prompt_template()
        runnable = prompt_template | chat_model
        runnable_with_history = RunnableWithMessageHistory(
            runnable,
            lambda _:self.get_session_history(session_id),
            input_messages_key="input",
            history_messages_key="history",
        )
        return runnable_with_history
    
    # 会話の処理関数
    def run_conversation(self, session_id: str, user_input: str):
        if self.runnable_with_history is None:
            self.runnable_with_history = self.gen_runnable(session_id)
        output = self.runnable_with_history.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": session_id}},
        )
        return output.content


In [24]:

# 1回目の会話
session_id = "123"
user_input = "うちの猫の名前は白子です。"
chat = Chat()
output = chat.run_conversation(session_id, user_input)
print(output)


素敵な名前ですね！白子ちゃんは元気ですか？


In [26]:
output = chat.run_conversation(
user_input= "うちの猫の名前を読んでください。",
session_id=123
)
print(output)


白子（しらこ）ですね。可愛い名前ですね。


In [28]:
output = chat.run_conversation(
user_input= "うちの猫の名前を使って、あだ名を考えて下さい",
session_id=123
)
print(output)

白ちゃん、シラちゃん、コちゃん、などいかがでしょうか？
